In [1]:
import os, shutil
import xml
import xml.etree.ElementTree as ET

In [2]:
import pandas as pd

# 1. zipファイル化と解凍
エクセルファイルの拡張子を .xlsx から .zip に変更し、解凍する。

In [3]:
# 拡張子の変更
src = './irasutoya_athletics.xlsx'  # 画像を取り出すエクセルファイル名
dst = src.replace('.xlsx', '.zip')  # コピー先のファイル名
shutil.copyfile(src, dst)           # 拡張子を .zip に変更して同階層にコピー

# 解凍
upk = os.path.splitext(src)[0]      # エクセルファイルと同名の解凍後フォルダ名
shutil.unpack_archive(dst, upk)     # .zip を解凍

# 2. xmlファイルの解析
解凍後フォルダ配下の ~/xl/media/ 内に入っている画像ファイル名と、シート上での位置（セル番地）との対応を解析します。  
対応関係は２つのファイルにまたがっているので、両方を見る必要があります。

x はシート作成順の連番だと思いますが、詳しくは分かりません。  
複数シートを含むエクセルファイルの場合は、各ファイルの中身を確認してみてください。

| 対応関係  | xmlファイル |
| :-- | :-- | 
| セル番地 ⇔ rID | ~/xl/drawings/drawing[x].xml |
| rID ⇔ 画像のパス | ~/xl/drawings/_rels/drawing[x].xml.rels |

In [4]:
def to_pretty_string(xml_path: str) -> str:
    """
    インデントの揃った見やすい文字列を取得

    Parameters
    ----------
    xml_path : str
        xmlファイル名

    Returns
    -------
    xml_str : str
        整形後の文字列
    """
    dom = xml.dom.minidom.parse(xml_path)
    xml_str = dom.toprettyxml()
    return xml_str

## 2.1. セル番地 ⇔ rID

### ファイル構成の確認
パースを進めていくにあたり、まずは構成を確認します。

In [5]:
drw = os.path.join(upk, 'xl/drawings/drawing1.xml') # 読み込むファイル名
print(to_pretty_string(drw))                        # 整形した文字列を出力

<?xml version="1.0" ?>
<xdr:wsDr xmlns:xdr="http://schemas.openxmlformats.org/drawingml/2006/spreadsheetDrawing" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main">
	<xdr:twoCellAnchor editAs="oneCell">
		<xdr:from>
			<xdr:col>1</xdr:col>
			<xdr:colOff>0</xdr:colOff>
			<xdr:row>2</xdr:row>
			<xdr:rowOff>1</xdr:rowOff>
		</xdr:from>
		<xdr:to>
			<xdr:col>1</xdr:col>
			<xdr:colOff>659230</xdr:colOff>
			<xdr:row>2</xdr:row>
			<xdr:rowOff>720001</xdr:rowOff>
		</xdr:to>
		<xdr:pic>
			<xdr:nvPicPr>
				<xdr:cNvPr id="2" name="図 1">
					<a:extLst>
						<a:ext uri="{FF2B5EF4-FFF2-40B4-BE49-F238E27FC236}">
							<a16:creationId xmlns:a16="http://schemas.microsoft.com/office/drawing/2014/main" id="{F81B7B08-0429-B1F9-B20D-7D0EEBB872AF}"/>
						</a:ext>
					</a:extLst>
				</xdr:cNvPr>
				<xdr:cNvPicPr>
					<a:picLocks noChangeAspect="1" noChangeArrowheads="1"/>
				</xdr:cNvPicPr>
			</xdr:nvPicPr>
			<xdr:blipFill>
				<a:blip xmlns:r="http://schemas.openxmlformats

### 必要情報の取得
各タグ \<xdr:xxx\> の xdr: の部分には、"{http://schemas.openxmlformats.org/drawingml/2006/spreadsheetDrawing}" が入ります。  
他にも \<a:xxx\> や \<r:xxx\> などがあり、繰り返し登場するので先に抜き出しておきます。  
バージョン違いなどで変わるかもしれないので、必要に応じて修正してください。

In [6]:
xdr = "{http://schemas.openxmlformats.org/drawingml/2006/spreadsheetDrawing}"
a = "{http://schemas.openxmlformats.org/drawingml/2006/main}"
r = "{http://schemas.openxmlformats.org/officeDocument/2006/relationships}"

In [7]:
drw_tree = ET.parse(drw)        # xmlのパーサ
drw_root = drw_tree.getroot()   # root として <xdr:wsDr> タグの部分を取得

次に、画像の位置情報を取得します。  
\<xdr:from\> 内は画像の開始位置情報です。

| タグ  | 意味 |
| :-- | :-- | 
| xdr:col | 画像の左上の列番号 |
| xdr:colOff | 画像のセル内での列方向オフセット |
| xdr:row | 画像の左上の行番号 |
| xdr:rowOff | 画像のセル内での行方向オフセット |

ここでは開始セル番地のみ取得しますが、必要に応じてほかの情報も取得してみてください。

In [8]:
col_elems = drw_root.findall(f'.//{xdr}twoCellAnchor/{xdr}from/{xdr}col')   # 画像位置の列番号の要素リストを取得
drw_cols = [int(elem.text) for elem in col_elems]                               # 値を整数に変換
drw_cols

[1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1]

In [9]:
row_elems = drw_root.findall(f'.//{xdr}twoCellAnchor/{xdr}from/{xdr}row')   # 画像位置の行番号の要素リストを取得
drw_rows = [int(elem.text) for elem in row_elems]                               # 値を整数に変換
drw_rows

[2, 2, 6, 4, 6, 4, 1, 1, 7, 7, 3, 3, 5, 5]

続いて、rID を取得します。

In [10]:
rid_elems = drw_root.findall(f'.//{xdr}twoCellAnchor/{xdr}pic/{xdr}blipFill/{a}blip')   # rIDの要素リストを取得
drw_rids = [elem.get(f'{r}embed') for elem in rid_elems]                                # rIDの値を取得
drw_rids

['rId1',
 'rId2',
 'rId3',
 'rId4',
 'rId5',
 'rId6',
 'rId7',
 'rId8',
 'rId9',
 'rId10',
 'rId11',
 'rId12',
 'rId13',
 'rId14']

ここまでの結果を DataFrame にまとめておきます。

In [11]:
drw_df = pd.DataFrame({'row': drw_rows, 'col': drw_cols, 'rid': drw_rids})
drw_df = drw_df.sort_values(['row', 'col']).reset_index(drop=True)
drw_df

,row,col,rid
0,1,1,rId8
1,1,2,rId7
2,2,1,rId1
3,2,2,rId2
4,3,1,rId12
5,3,2,rId11
6,4,1,rId4
7,4,2,rId6
8,5,1,rId14
9,5,2,rId13


## 2.2. rID ⇔ 画像のパス

### ファイル構成の確認
パースを進めていくにあたり、まずは構成を確認します。

In [12]:
rel = os.path.join(upk, 'xl/drawings/_rels/drawing1.xml.rels') # 読み込むファイル名
print(to_pretty_string(rel))                        # 整形した文字列を出力

<?xml version="1.0" ?>
<Relationships xmlns="http://schemas.openxmlformats.org/package/2006/relationships">
	<Relationship Id="rId8" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image8.png"/>
	<Relationship Id="rId13" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image13.png"/>
	<Relationship Id="rId3" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image3.png"/>
	<Relationship Id="rId7" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image7.png"/>
	<Relationship Id="rId12" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image12.png"/>
	<Relationship Id="rId2" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/image" Target="../media/image2.png"/>
	<Relationship Id="rId1" Type="http://schemas.openxmlformats.org/o

### 必要情報の取得
画像位置の取得と同様に処理します。

In [13]:
xmlns = "{http://schemas.openxmlformats.org/package/2006/relationships}"

In [14]:
rel_tree = ET.parse(rel)        # xmlのパーサ
rel_root = rel_tree.getroot()   # root として <Relationships> タグの部分を取得

In [15]:
rel_elems = rel_root.findall(f'.//{xmlns}Relationship')                     # 対応関係の要素リストを取得
rel_rids = [elem.get('Id') for elem in rel_elems]                           # rIDのリストを取得
rel_imgs = [os.path.basename(elem.get('Target')) for elem in rel_elems]     # ファイル名のリストを取得

結果を DataFrame にまとめます。

In [16]:
rel_df = pd.DataFrame({'rid': rel_rids, 'name': rel_imgs})
rel_df

,rid,name
0,rId8,image8.png
1,rId13,image13.png
2,rId3,image3.png
3,rId7,image7.png
4,rId12,image12.png
5,rId2,image2.png
6,rId1,image1.png
7,rId6,image6.png
8,rId11,image11.png
9,rId5,image5.png


## 2.3. 対応関係の整理
ここまでで、セル番地 ⇔ rID と rID ⇔ 画像のパス の対応関係を整理できました。  
最後に、セル番地 ⇔ 画像のパス の対応関係をまとめます。

rID と画像ファイル名の番号がずれる場合ってあるのでしょうか？  
もしないのであれば、.rels ファイルは解析しなくても良いですね。

In [17]:
res_df = pd.merge(left=drw_df, right=rel_df, how='inner', on='rid')
res_df

,row,col,rid,name
0,1,1,rId8,image8.png
1,1,2,rId7,image7.png
2,2,1,rId1,image1.png
3,2,2,rId2,image2.png
4,3,1,rId12,image12.png
5,3,2,rId11,image11.png
6,4,1,rId4,image4.png
7,4,2,rId6,image6.png
8,5,1,rId14,image14.png
9,5,2,rId13,image13.png


# おまけ. シート内のテキスト情報をもとにリネーム
画像が表に並べられているような場合、見出し情報をもとにリネームすると整理が楽にできます。  
xml を解析して取得したセル番地は、1 始まりです。  
pandas の read_excel() では、index が 0 始まりであることに注意してリネームまでしてみます。

In [18]:
df = pd.read_excel(src, index_col=0)
df

,女性,男性
種目,,
短距離走,NaN,NaN
ハードル,NaN,NaN
長距離走,NaN,NaN
走り幅跳び,NaN,NaN
三段跳び,NaN,NaN
走り高跳び,NaN,NaN
棒高跳び,NaN,NaN


In [19]:
rnm_df = res_df.copy()
rnm_df['event'] = df.index[rnm_df.row - 1]                                              # 行見出しから種目を抽出
rnm_df['gender'] = df.columns[rnm_df.col - 1]                                           # 列見出しから性別を抽出
rnm_df['rename'] = rnm_df.apply(lambda ser: f'{ser.event}_{ser.gender}.png', axis=1)    # 一例として 種目_性別.png 
rnm_df

,row,col,rid,name,event,gender,rename
0,1,1,rId8,image8.png,短距離走,女性,短距離走_女性.png
1,1,2,rId7,image7.png,短距離走,男性,短距離走_男性.png
2,2,1,rId1,image1.png,ハードル,女性,ハードル_女性.png
3,2,2,rId2,image2.png,ハードル,男性,ハードル_男性.png
4,3,1,rId12,image12.png,長距離走,女性,長距離走_女性.png
5,3,2,rId11,image11.png,長距離走,男性,長距離走_男性.png
6,4,1,rId4,image4.png,走り幅跳び,女性,走り幅跳び_女性.png
7,4,2,rId6,image6.png,走り幅跳び,男性,走り幅跳び_男性.png
8,5,1,rId14,image14.png,三段跳び,女性,三段跳び_女性.png
9,5,2,rId13,image13.png,三段跳び,男性,三段跳び_男性.png


In [20]:
for from_, to_ in zip(rnm_df['name'], rnm_df['rename']):
    # コピー元のファイル名
    f = os.path.join(upk, f'xl/media/{from_}')
    # コピー先のファイル名
    t = f'{upk}_images/{to_}'
    os.makedirs(os.path.dirname(t), exist_ok=True)    # ディレクトリが存在しなければ作成
    # コピー
    shutil.copy(f, t)